In [1]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 68.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 79.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 63.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 100.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Prepar

In [2]:
import ktrain
from ktrain import text

In [3]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [4]:
DATA_PATH = '/content/train.csv'
NUM_WORDS = 500000
MAXLEN = 500
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv( DATA_PATH,
                                                                    'text',
                                                                      label_columns = 'sentiment',
                                                                      val_filepath=None, # if None, 10% of data will be used for validation
                                                                      max_features=NUM_WORDS, maxlen=MAXLEN,
                                                                      ngram_range=1)

detected encoding: utf-8 (if wrong, set manually)
['not_sentiment', 'sentiment']
       not_sentiment  sentiment
2613             1.0        0.0
5640             1.0        0.0
14619            0.0        1.0
17105            0.0        1.0
16850            0.0        1.0
['not_sentiment', 'sentiment']
       not_sentiment  sentiment
4607             1.0        0.0
198              1.0        0.0
10096            1.0        0.0
11835            1.0        0.0
16139            0.0        1.0
language: en
Word Counts: 84599
Nrows: 22500
22500 train sequences
train sequence lengths:
	mean : 234
	95percentile : 602
	99percentile : 916
x_train shape: (22500,500)
y_train shape: (22500, 2)
Is Multi-Label? False
2500 test sequences
test sequence lengths:
	mean : 226
	95percentile : 559
	99percentile : 885
x_test shape: (2500,500)
y_test shape: (2500, 2)


In [5]:
model = text.text_classifier('logreg', (x_train,y_train), preproc=preproc)

Is Multi-Label? False
compiling word ID features...
maxlen is 500
done.


In [6]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test))

In [7]:
learner.fit_onecycle(1e-4, 1)



begin training using onecycle policy with max lr of 0.0001...
704/704 [==============================] - 6s 7ms/step - loss: 0.6127 - accuracy: 0.7603 - val_loss: 0.5513 - val_accuracy: 0.8308


In [8]:
from sklearn.metrics import roc_auc_score
y_pred = learner.model.predict(x_test, verbose=0)
score = roc_auc_score(y_test, y_pred)
print("\n ROC-AUC score: %.6f \n" % (score))


 ROC-AUC score: 0.905539 



In [9]:
predictor = ktrain.get_predictor(learner.model , preproc)

In [18]:
sents = x_test[0]

In [26]:
data = [ 'This movie was horrible! The plot was boring. Acting was okay, though.',
         'The film really sucked. I want my money back.',
        'What a beautiful romantic comedy. 10/10 would see again!']

In [27]:
predictor.predict(data)

['sentiment', 'sentiment', 'not_sentiment']

In [31]:
import pandas as pd
df = pd.read_csv('/content/train.csv')
df.head()

,text,sentiment
0,For a movie that gets no respect there sure ar...,0
1,Bizarre horror movie filled with famous faces ...,0
2,"A solid, if unremarkable film. Matthau, as Ein...",0
3,It's a strange feeling to sit alone in a theat...,0
4,"You probably all already know this by now, but...",0


In [32]:
selected_rows = df[12000:13000]
selected_text = selected_rows['text']
selected_sentiment = selected_rows['sentiment']
selected_text ,selected_sentiment

(12000    So, neighbor was killing neighbor. Reminds me ...
 12001    The first time i saw it i got half of it but i...
 12002    This has to be one of the best comedies on the...
 12003    This would have to be one of the funniest TV s...
 12004    I gave this a 10 out of 10 points. I love it s...
                                ...                        
 12995    I just finished watching this film and WOW was...
 12996    The script for this TV soap opera is so bad th...
 12997    In 1987, John Hughes wrote and directed 'Plane...
 12998    The NSA, CIA, FBI, FSB and all other snoop age...
 12999    What on earth has become of our dear Ramu? Is ...
 Name: text, Length: 1000, dtype: object,
 12000    0
 12001    0
 12002    0
 12003    0
 12004    0
         ..
 12995    1
 12996    1
 12997    1
 12998    1
 12999    1
 Name: sentiment, Length: 1000, dtype: int64)

In [33]:
y_pred = predictor.predict(selected_text.tolist(), verbose=0)

In [34]:
mapping = {'not_sentiment':0 , 'sentiment':1}
y_pred_d = list(mapping[val] for val in y_pred )

In [35]:
#Classification report for bag of words 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
report =classification_report(selected_sentiment,y_pred_d)#,target_names=['Positive','Negative'])
print(report)

              precision    recall  f1-score   support

           0       0.86      0.82      0.84       500
           1       0.83      0.87      0.85       500

    accuracy                           0.84      1000
   macro avg       0.85      0.84      0.84      1000
weighted avg       0.85      0.84      0.84      1000



In [36]:
predictor.save('/content/logistic_regressor_model')

In [39]:
print('model saved')

model saved
